# 机翼流固耦合模拟（浸入边界方法）

AERO-Suite里的**AERO-F**是流体仿真模拟器，这里是详细的[**AERO-F**教程](https://bitbucket.org/frg/aero-f/downloads/AERO-F.pdf)。
**AERO-S**是固体仿真模拟器，这里是详细的[**AERO-S**教程](https://bitbucket.org/frg/aero-s/downloads/AERO-S.pdf)。AERO-Suite对流固耦合的仿真采用的是 [分区耦合（Partitioned）方式](../../AGARD_ALE/simulations/AGARD_ALE_FSI_README.ipynb)。

本案例是使用浸入边界方法（EBM）对机翼的气动弹性力学的仿真模拟。[AGARD机翼](https://ntrs.nasa.gov/citations/19880001820) 是美国国家航空和航天局（NASA）用于测试空气动弹性的一个标准模型，它由多块壳单元构成。
本次仿真是AGARD机翼受损后的气动弹性模拟，即机翼的杨氏模量和扭转模量都更低。本次仿真的文件都在 `Wing/AGARD_EBM` 文件夹中，
在这个文件中一般有3个子文件夹：`sources`, `simulations`和`data`。
- `sources`文件夹一般存放网格相关的源文件
- `simulations`文件夹一般存放仿真模拟的输入文件
- `data`文件夹一般存放仿真模拟过程中生成的一些文件。

以下是本次仿真模拟的详细步骤。
在开始本次仿真模拟前，我们需要安装相关的软件，请参加安装教程,比如[如何在北京大学未名一号上安装AERO-Suite](../../../Install/Install_PKU.ipynb)。
如果我们之前已经安装好相关软件，我们可以读取 [.bashrc_frg](../../../Install/bashrc_frg) 刷新当前shell环境，载入各个软件的命令
```shell
source ~/.bashrc_frg 
```

## 网格文件

流体计算区域是一个平行六面体，我们在 `sources` 文件夹中，用**python**生成指数拉伸的网格

```shell
python generateMesh.py
```
我们能在 [generateMesh.py](../sources/generateMesh.py) 中的 `agard3D()` 函数里，指定 `x`, `y`, `z` 数组，并指定`z`底部，`z`顶部，`x`底部，`x`顶部，`y`底部，`y`顶部的边界名称，生成结构网格 `fluid.top`。

固体网格 [Structure.top](../sources/Structure.top)是三角形壳单元。值得注意的是我们只在后处理画图时使用这个网格。仿真模拟中需要的固体信息都在固体的输入文件 [StructureFile.FSI](./StructureFile.FSI)，用**AERO-S**可以处理固体输入文件[StructureFile.FSI](./StructureFile.FSI)生成相应的固体网格文件 `Structure.top`：

```shell
aeros -t StructureFile.FSI 
```
这里我们加了命令参数 `-t` ，在 [StructureFile.FSI](./StructureFile.FSI) 里仿真的名字为 Structure， 因而生成的文件是 `Structure.top`。

对于使用浸入边界方法的仿真模拟，我们需要输入流体求解器表示固体形状的浸入表面 [embeddedSurface.top](../sources/embeddedSurface.top)，这个浸入表面可以和固体表面网格相同，也能与之不同。本案例中，浸入表面表示了机翼的形状，而固体网格是用壳单元对机翼的简化模型，因而他们并不相同。

我们可以用**xp2exo**把 `.top` 网格文件转化为 `.exo` 文件进行可视化，这里我们展示了流体网格、浸入表面、和固体网格:
<img src="../../../Figs/AGARD_EBM.png" width="800" />


# 网格前处理

我们首先使用**partnmesh**软件，能把流体网格划分35个区域，使用35个核进行并行运算。这可以在`AGARD_EBM`文件里用以下命令行完成

```shell
partnmesh sources/fluid.top 35
```

这会在`sources`文件夹中，生成有分区标注的`fluid.top.dec.8`文件。

如前面所言，对于使用浸入边界方法的仿真模拟，我们需要使用**matcher**去匹配浸入表面 [embeddedSurface.top](../sources/embeddedSurface.top) 里标注了是 *Moving* 的边界单元的格点（比如StickMovingSurface），和固体表面的网格 [matcher.top](../sources/matcher.top)。这可以在`AGARD_EBM`文件里用以下命令行完成

```shell
matcher sources/embeddedSurface.top sources/matcher.top -e 1 -p 8 -l 2  -output data/fluidmodel
```
这里 `-e 1 -p 8 -l 2` 是关于最大匹配距离，并行线程数和迭代次数的一些参数。这会在 `data` 文件夹里生成流体模拟使用的匹配文件 `fluidmodel.match.fluid`、固体模拟使用的匹配文件 `fluidmodel.match.fem` 和供可视化的 `fluidmodel.match.top` 文件。其中匹配文件 `fluidmodel.match.fluid`里是匹配的浸入表面上的格点数目共2788个，然后是格点在浸入表面中的编号，然后是它们到固体表面网格的距离向量； `fluidmodel.match.fem`里，首先是匹配的浸入表面上的格点数目共2788个，然后是每个匹配了的浸入表面格点的匹配信息 $n,\xi,\eta,n_x,n_y,n_z$， 其中$n$是流体格点匹配的固体表面网格 [matcher.top](../sources/matcher.top)里三角形有限元对应的编号，$\xi,\eta$ 是对应的匹配点在三角形有限元里的自然坐标，$n_x,n_y,n_z$ 是流体格点到固体表面匹配点的距离向量。用这些信息我们AERO-Suite能实现流体、固体求解器对于流体对固体作用力、固体表面位移的交互。

之后我们需要用**sower**去生成各个区域网格的详细信息，把这些信息文件储存在`data`文件夹中，这可以在`AGARD`文件夹里用以下命令行完成

```shell
sower -fluid -mesh sources/fluid.top -match data/fluidmodel.match.fluid  -dec sources/fluid.top.dec.35 -cpu 35 -cluster 35 -output data/fluidmodel
```

这会在 `data` 文件夹中生成以 `fluidmodel` 为前缀的文件，包括各区域的网格信息比如 `fluidmodel.msh1`，标注信息比如 `fluidmodel.dec1`，匹配信息比如 `fluidmodel.fluid.match1`， 各35个文件，以及CPU和分区的映射 `fluidmodel.35cpu` 和各个分区的连接信息 `fluidmodel.con`。


## 流固耦合计算

### 流体部分

对于所有流固耦合问题，我们首先假设固体是不动的，对流体做模拟仿真，然后用流体仿真的结果作为流固耦合模拟仿真流体的初始状态。对于本次模拟，这个流体仿真的输入文件是 [simulations/FluidFile.Steady](./FluidFile.Steady)。这个流体仿真会把重启文件存放在 `postpro.Steady` 文件夹里。本次案例我们不再讲解这个定常流的仿真模拟，不熟悉的读者可以参见之前的[NACA EBM 教程](../../../NACA/EBM/simulations/NACA.EBM.Steady.ipynb)。


对于流固耦合的模拟仿真，我们的流体部分仿真输入文件是 [simulations/FluidFile.FSI](./FluidFile.FSI)，这里我们首先指定求解问题的类型，本次仿真使用浸入边界方法（BodyFitted），是有量纲的（Dimensional）非定常的气动弹性力学（UnsteadyAeroelastic）的模拟。

```shell
under Problem {
  Type = UnsteadyAeroelastic;
  Mode = Dimensional; 
  Framework = Embedded;
}
```

然后指定网格相关的文件（Input），它们是网格分区预处理的生成文件，一般在`data`文件夹里面。这里可以给出这些网格相关文件的前缀，**AERO-F**会自动去查找这些文件。他们包括格点信息 `.msh`、网格分区标注信息 `.dec`, 网格连接信息 `.con`、CPU和分区映射的信息 `.cpu`。值得注意的是对流固耦合问题我们也要提供流固耦合边界匹配的信息（Matcher）和浸入边界（EmbeddedSurface）。我们这里也指定了流体的初始状态（Solution）是定常流模拟的结果。
```shell
under Input {
  GeometryPrefix = "../data/fluidmodel";
  
  Matcher = "../data/fluidmodel.match.fluid";
  EmbeddedSurface = "../sources/embeddedSurface.top";
  Solution = "references.Steady/Solution.bin";
}
```

然后指定仿真模拟中间的输出。这里包括我们感兴趣的一些量（Postpro）和用于重启模拟的文件（Restart），其中频率（Frequency）指定每多少步输出一次，如果频率是0，只在最后模拟结束时输出。如果我们要输出固体表面的量，比如压力系数（EmbeddedSurfacePressureCoefficient），对于两侧都有流体的固体（比如降落伞）我们需要用 PostprocessSides 指定输出的是哪一侧的量，其中 1 表示网格外法线一侧。对于只有一侧有流体的固体，我们需要固体的外法向指向流体一侧。
为了检查固体网格外法线的朝向，我们可以先用**xp2exo**把固体 `.top` 输入文件转化为 `.exo`文件，然后用**Paraview**打开，用Filters-Alphabetical-Normal Glyphs对外法线进行可视化查看。

```shell
under Output {
  under Postpro {
    Prefix = "results.FSI/";
    LiftandDrag = "../postpro.FSI/LiftandDrag.out";
    Force = "../postpro.FSI/Force.out";
    Mach = "Mach.bin";
    Pressure = "Pressure.bin";
    EmbeddedSurfaceDisplacement = "../postpro.FSI/EmbeddedSurfaceDisplacement.xpost";
    EmbeddedSurfacePressureCoefficient = "../postpro.FSI/EmbeddedSurfacePressureCoefficient.xpost";
    Frequency = 100;
    PostprocessSides = 1;
  }
  under Restart {
    Prefix = "references.FSI/"; 
    Solution = "Solution.bin";
    RestartData = "Restart.data";
    Position = "Position.data";
    EmbeddedPosition = "EmbeddedPosition.data";
    Frequency = 0;
  }
}
```

然后指定边界条件。这里包括远场边界条件（Inlet）和固体表面边界条件（Wall）。

```shell
under BoundaryConditions {
  under Inlet {
    Mach = 0.97;
    Alpha = 7.5; 
    Beta = 0.0;
    Density = 0.61115933E-7;  // slugs / in^3
    Pressure = 6.0;           // psi*12 ([slugs/(in-s^2)])
  }
}
```

然后指定我们要求解的方程以及相关的参数，比如Euler方程或者Navier Stokes方程，以及湍流闭包模型。

```shell
under Equations {  
  Type = Euler;   
  under FluidModel[0] {
    Fluid = PerfectGas;
    under GasModel {
      SpecificHeatRatio = 1.4;
    }
  }
}
```

然后指定空间离散格式。

```shell
under Space {
  under NavierStokes {
    Flux = Roe;
    Reconstruction = Linear;
    AdvectiveOperator = FiniteVolume;
    Limiter = VanAlbada;
    Gradient = LeastSquares;
    Dissipation = SecondOrder;
    Beta = 0.3333333333333333;
    Gamma = 1.0;
  }
  under Boundaries {
    Type = StegerWarming;
  }
}
```

最后指定时间离散格式，包括CFL条件，显、隐格式，牛顿迭代的参数等。

```shell
under Time {
  Type = Implicit;
  MaxIts = 5000;
  under CflLaw {
     Cfl0 = 1.0e+2;
     CflMax = 1.0e+2;
  }
  under Implicit {
    Type = RungeKutta2;
    MatrixVectorProduct = FiniteDifference;
    under Newton { 
      MaxIts = 5; 
      Eps = 1e-05;
      FailSafe = AlwaysOn;
      under LinearSolver {
        under NavierStokes {
          Type = Gmres;
          MaxIts = 100;
          KrylovVectors = 100;
          Eps = 1.0e-4;
          //Output = "stdout";
          under Preconditioner { 
            Type = Ras; 
            Fill = 0; 
          }
        }
      }
    }
  }
}
```

最后指定浸入边界方法的参数，包括求解半黎曼问题的位置（TypeHalfRiemannProblem），选择在控制体积边界（Surrogate）能避免外插更加稳定，但是只有一阶精度。 把流体速度投影到一维求解半黎曼问题使用的向量（RiemannNormal）等。

```shell
under EmbeddedFramework {
  TypeHalfRiemannProblem = Surrogate;
  RiemannNormal = Structure;  
}
```



### 固体部分
我们的固体部分的仿真输入文件是 [simulations/Structure.FSI](./Structure.FSI)，
这里我们首先指定固体仿真的名字，固体网格格点和有限元的名字

```shell
CONTROL
Structure
1
StructureNodes
StructureElements
```

然后指定求解问题的类型，本次仿真是气动弹性力学的模拟（AERO）。AERO-Suite采用的是分区耦合（Partitioned），我们需要选择耦合方式，`A6 0.5 0.125` 是流体求解器采用隐式格式、固体求解器采用隐式中点法则的二阶精度的交错求解算法。`C0 0.5 0.375` 是流体求解器采用隐式或显示格式、固体求解器采用隐式中点法则的二阶精度的交错求解算法。我们需要提供流固耦合边界匹配的信息（MATCHER）。

```shell
AERO
A6 0.5 0.125
MATCHER "../data/fluidmodel.match.fem"
```

然后指定时间积分，固体时间积分一般是NEWMARK格式，指定格式参数（MECH），我们采用中点法则。
以及固体求解器时间步长（1.0e-4），以及起始时刻。

```shell
DYNAMICS
NEWMARK
MECH          0.25  0.50
TIME          0.0   0.5e-4  0.1
```

然后对于隐式时间积分，指定线性方程组求解器。

```shell
STATIC
MUMPS PIVOT
MUMPS_ICNTL 14 200
```

有时候线性方程组不一定可逆，比如在有刚体模式的时候，我们在矩阵分解过程会把小的置换值（pivots）换成这个容差。

```shell
TRBM
1.0E-09
```

指定是非线性模拟， 包括几何非线性和潜在的材料非线性

```shell
NONLINEAR
```

然后指定计算结束输出固体的质量、质心、转动惯量等

```shell
MASS
```

然后指定输出量，输出量的文件名，以及输出频率

```shell
OUTPUT
GDISPLAC "postpro.FSI/structure.GDISPLAC.xpost" 100
OUTPUT6
GDISPLAC "postpro.FSI/structure.GDISPLAC.111" 1 111
GDISPLAC "postpro.FSI/structure.GDISPLAC.116" 1 116
GDISPLAC "postpro.FSI/structure.GDISPLAC.121" 1 121
```


最后指定固体网格以及材料参数。[StructureFile.include](./StructureFile.include) 和 [StructureFile.composite](./StructureFile.composite) 是固体网格，首先是格点（NODES），然后是有限元（TOPOLOGY），这里我们使用的是三角形壳单元，接下来是有限元属性（ATTRIBUTES）列表，包括基本材料属性编号（比如杨氏模量）、复合材料属性编号、复合材料坐标系框架标编号等。
接下来是基本材料属性（MATERIALS）列表，复合材料属性（COMPOSITE）列表和复合材料坐标系框架（CFRAME）。
最后是Dirichlet边界条件(DISPLACEMENTS)，指定格点、相应的自由度的值。

```shell
INCLUDE "StructureFile.include"
INCLUDE "StructureFile.composite"
END
```


我们可以在`simulations`中，用以下命令行在8个核上运行**AERO-F**，以及1个核上运行**AERO-S*。

```shell
mpirun -n 35 aerof.opt FluidFile.FSI : -n 1 aeros StructureFile.FSI |& tee log.FSI.out
```

我们感兴趣的一些量，比如流体的压强（比如Pressure.bin1）和马赫数（Mach.bin1）都存在 `results.FSI` 文件夹, 像压强和马赫数这些流体状态的量，都是每一个分区上的数据，因此各有35个文件。值得注意的是像压力系数 (EmbeddedSurfacePressureCoefficient) 这样的状态量，只存在于翼型表面的格点，像升力阻力（LiftandDrag）它们都不需要后处理，我们存在了 `postpro.FSI` 文件夹。
另外一些用于重启模拟的文件，都存在 `references.FSI` 文件夹。


## 结果后处理

我们需要首先用**sower**把**AERO-F**并行输出的结果拼接起来。我们可以在`simulations`中，用以下命令行拼接关于压强和马赫数的结果。

```shell
sower -fluid -merge -con ../data/fluidmodel.con -mesh ../data/fluidmodel.msh -result results.FSI/Pressure.bin -output postpro.FSI/Pressure

sower -fluid -merge -con ../data/fluidmodel.con -mesh ../data/fluidmodel.msh -result results.FSI/Mach.bin -output postpro.FSI/Mach
```

对于流体输出，拼接起来的文件或是浸入边界上量的文件是在 `postpro.FSI` 文件夹里面后缀为 `.xpost` 的文件，为了使用**paraview**做后处理，我们需要用**xp2exo**把后缀为 `.xpost` 的文件转化为后缀为 `.exo` 的文件；对于固体输出，我们也需要用**xp2exo**把结果转化为后缀为 `.exo` 的文件。我们可以在 `simulations` 中，用以下命令行进行转换

```shell
xp2exo ../sources/fluid.top     fluid_solution.FSI.exo     postpro.FSI/Pressure.xpost postpro.FSI/Mach.xpost 

xp2exo ../sources/embeddedSurface.top  embeddedSurface.FSI.exo  postpro.FSI/EmbeddedSurfaceDisplacement.xpost postpro.FSI/EmbeddedSurfacePressureCoefficient.xpost

xp2exo ../sources/Structure.top  structure_solution.FSI.exo  postpro.FSI/structure.GDISPLAC.xpost 
```


我们可以用[**paraview**](https://www.paraview.org/tutorials/)读入 `fluid_solution.FSI.exo` 、 `embeddedSurface.FSI.exo ` 和 `structure_solution.FSI.exo` 进行可视化。
这里我们用**paraview**展示了马赫场和AGARD机翼的形变以及上面的压力系数
<img src="../../../Figs/AGARD_EBM_FSI.png" width="300" />


## bash脚本运行

对于网格划分的前处理，我们可以把这些命令行写在 `AGARD_EBM` 文件夹里的脚本文件里，比如[preprocess.sh](../preprocess.sh)。我们可以用以下命令行运行脚本

```shell
bash preprocess.sh
```


对于运行**AERO-F**的定常流模拟以及结果的后处理，我们可以把这些命令行写在 `simulations` 文件夹里的脚本文件里。我们可以用以下命令运行脚本

```shell
bash run.Steady.sh
bash postprocess.Steady.sh
```

对于运行**AERO-F**和**AERO-S**的流固耦合模拟以及结果的后处理，我们可以把这些命令行写在 `simulations` 文件夹里的脚本文件里。我们可以用以下命令运行脚本

```shell
bash run.FSI.sh
bash postprocess.FSI.sh
```

为了在集群上提交作业，我们需要用 [sbatch命令](https://hpc.pku.edu.cn/_book/guide/slurm/sbatch.html) 将作业提交到计算节点上执行，在`simulations`文件夹里，我们可以编写作业脚本文件 [Sbatch.FSI.sh](./Sbatch.FSI.sh) 申请计算节点，进行计算。可以用以下命令行提交作业

```shell
sbatch Sbatch.FSI.sh
```



